
<center>
    <img src="images/logo-2-1024.png">
</center>


# Cosa è YouTube-PublicSentiment

**YouTube-PublicSentiment** è uno strumento per effettuare sentiment analysis, entity sentiment analysis e emotion recognition sui commenti lasciati dal pubblico sotto un determinato video sulla piattaforma YouTube.

## Chi è il Target
**YouTube-PublicSentiment** è pensato per un pubblico ampio, inclusi content creator, aziende e qualsiasi utente curioso desideroso di comprendere il sentimento del pubblico verso i video su YouTube. Non è necessario essere il proprietario del video per utilizzare lo strumento.

## Come é nato il progetto
L'idea di realizzare questo strumento è nata dalla curiosità personale di conoscere il sentimento del pubblico nei confronti degli argomenti trattati in un video e di tutto ciò che lo circonda, e di come questo possa variare nel tempo, in seguito a eventi specifici avvenuti sia all'interno che all'esterno della piattaforma YouTube o per nessun motivo apperente.

I commenti sotto i video spesso rivelano una vasta gamma di opinioni, emozioni e reazioni, offrendo una comprensione più profonda della percezione del pubblico verso il contenuto del video e tutto ciò che lo circonda, comprese persone, organizzazioni, prodotti e beni di consumo.

Leggere solo alcuni commenti a campione potrebbe non essere rappresentativo del reale sentimento del pubblico, soprattutto considerato l'elevato numero di commenti che alcuni video possono ricevere.

Quella che inizialmente era semplice curiosità può rappresentare un'utilità significativa per i content creator e le aziende. Un'analisi dettagliata dei commenti può fornire ai content creator preziose informazioni per migliorare i loro contenuti e capire meglio il proprio pubblico, così come può essere utilizzata da aziende per comprendere il sentimento nei confronti dei loro prodotti e/o servizi.

## Come funziona?
YouTube-PublicSentiment monitora la sezione commenti di un video di cui è stato fornito in input il link ed effettua la sentiment analysis e l'entity analysis utilizzando Google Cloud Natural Language AI. Le informazione ottenute comprendono:


    

- per commento:
    - `sentiment_score`: numero compreso tra **\[ -1.0 , 1.0 \]** 
    - `sentiment_magnitude`: numero compreso tra **\[ 0 , +inf )**, indica quanto contenuto emotivo è presente all'interno del commento

- per singola entita estratta dal commento:
    - `entity_name`: nome rappresentativo per l'entità
    - `entity_type`: tipo di entità
    - `entity_salience`: numero compreso tra **\[ 0 , 1.0 \]**, indica quanto è importante l'entità all'interno del commento
    - `sentiment_score`: numero compreso tra **\[ -1.0 , 1.0 \]** 
    - `sentiment_magnitude`: numero compreso tra **\[ 0 , +inf )**, indica quanto contenuto emotivo è presente all'interno dell'entità

Successivamente, il testo del commento viene passato al modello di machine learning preaddestrato di **Hugging Face**: `bhadresh-savani/bert-base-go-emotion`. Questo modello è stato addestrato sul dataset `google-research-datasets/go_emotions`.

Il dataset go_emotions contiene 58.000 commenti di Reddit attentamente curati e etichettati per 27 categorie di emozioni, oltre a una categoria neutra. Questo rende il modello altamente efficace nel rilevare una vasta gamma di emozioni nei testi.

Per ogni emozione, avremo quindi uno score numerico float che indica la percentuale rilevata di ciascuna di esse.
Questo consente di ottenere una valutazione dettagliata delle emozioni espresse nei commenti, migliorando la comprensione delle reazioni emotive degli utenti.

# Dimostrazione Live!

## Dashboard

Analisi del seguente video: 
- <a href="https://www.youtube.com/watch?v=AXlL_IzEN8c">Trump | Assassination Attempt Minute by Minute: ABC News  Special</a>

<img src="images/Dashboard-r1.png">

<img src="images/Dashboard-r2.png">

<img src="images/Dashboard-r3.png">

<img src="images/Dashboard-r4.png">

<img src="images/Dashboard-r5.png">

## API utilizzate
- <a href="https://developers.google.com/youtube/v3">YouTube Data API</a> 
- <a href="https://cloud.google.com/natural-language?hl=it">Google Cloud Natural Language API</a>

<center>
  <img src="images/googleApiMeme.png" width="450">
</center>

## Architettura

###  Pipeline complessiva
<br>

<center>
    <img src="images/pipeline.svg" width="700">
</center>

## Breakdown delle singole componenti

<center>
    <img src="images/Docker.png" width="15%">
</center>

Per la containerizzazione di ciascun servizio:

  - Ambienti coerenti e isolati attraverso la containerizzazione
  - Distribuzione, scalabilità e gestione delle applicazioni semplificate
  - Utilizzo leggero ed efficiente delle risorse
  - Un modo semplice per gestire applicazioni multi-container attraverso Docker Compose

### Python Script

I seguenti passaggi sono ripetuti con una frequenza di 30 secondi per evitare la formazione di stream batch troppo grandi su Spark:

   - Effettua la chiamata all'API di YouTube per recuperare i commenti del video da analizzare.
   - Processa i commenti, rimuovendo eventuali tag HTML dal testo e selezionando solo i campi utili.
   - Invia i dati a Logstash per farli avanzare lungo la pipeline.


<center>
    <img src="images/LogstashFullLogo.png" width="30%"> 
</center>

Per la parte di Data ingestion:

   - Si integra perfettamente con Elasticsearch e Kibana.
   - Ha un'ottima usabilità.
   - Gestisce in modo efficiente grandi volumi di dati.

<center>
    <img src="images/Kafka.png" width="20%">
</center>

In KRaft Mode per la parte di Data Streaming:

   - Capace di gestire grandi volumi di dati con bassa latenza.
   - Non richiede ZooKeeper, con ridotta complessità e overhead operativi.
   - Assicura un migliore supporto per future release, allineato alla roadmap di Kafka.
   - Memorizza i dati in un unico topic: youtube-comments.

          
<div style="display: flex; justify-content: center; align-items: center;">
    <img src="images/Spark.png" alt="Immagine 1" style="max-width: 25%; height: auto;">
    <img src="images/GoogleCloud.png" alt="Immagine 2" style="max-width: 25%; height: auto;">
    <img src="images/HuggingFace.png" alt="Immagine 3" style="max-width: 25%; height: auto;">
</div>


### Spark con Google Cloud Natural Language AI e Hugging Face

   - Utilizza Google Cloud Natural Language API per eseguire la sentiment analysis e l'entity analysis.
   - Applica il modello BERT preaddestrato.
   - Espande la lista di entità.
   - Scrive in due indici separati per distinguere tra dati con e senza duplicati.

<center>
    <img src="images/ElasticsearchFullLogo.svg" width="30%">
</center>

Per la parte di Data Indexing:

   - Offre funzionalità di ricerca e analisi in tempo reale veloci e potenti.
   - Integrazione completa con Kibana nel ELK Stack.
   - Gestisce facilmente grandi volumi di dati.
   - Riceve dati da Apache Spark in due indici distinti: youtubecomments e youtubecomments_without_duplicates.

<center>
    <img src="images/Kibana.png" width="20%">
</center>

Per la parte di Data Visualization:

   - Integrazione completa con Elasticsearch nel ELK Stack.
   - Interfaccia utente user-friendly.
   - Grafici intuitivi e moderni.

## Possibili miglioramenti

- Aggiunta di un'interfaccia grafica per facilitare l'inserimento del link del video di cui analizzare i commenti.
- Analisi delle risposte ai commenti, non solo dei commenti top-level.
- Transizione a Language.v2 per supportare l'analisi multilingua quando il servizio di extract_entity_sentiment verrà incluso.
- Implementazione di un modello di emotion recognition con un maggiore supporto multilingua.

# Grazie per l'attenzione